# 사전 준비

In [ ]:
!pip install transformers

**경고 메시지 끄기**

In [2]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

**사전학습모델과 토크나이저(KcELECTRA, KoGPT2) 불러오기**

In [3]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, GPT2LMHeadModel, PreTrainedTokenizerFast

ELECTRA_tokenizer = ElectraTokenizer.from_pretrained("beomi/KcELECTRA-base", padding_side='left')
ELECTRA_model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base")   # 비속어 감지의 사전학습 모델로는 KcELECTRA 사용

GPT2_tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>',       # 문장 시작토큰
    eos_token='</s>',       # 문장 마지막토큰
    unk_token='<unk>',      # 어휘에 없는 토큰
    pad_token='<pad>',      # 크기 맞추기 토큰
    mask_token='<mask>',     # 마스킹 토큰
)
GPT2_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')   # 챗봇의 사전학습 모델로는 KoGPT2 사용

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.


...

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

# 데이터 불러오기

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import urllib.request

In [5]:
curse_data = pd.read_table("/content/drive/Othercomputers/내 컴퓨터/Curse_Detection_Chatbot/Curse-detection-data/dataset.txt", 
                           names=["text", "label"], sep="|", header=None)
curse_data

,text,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1
...,...,...
5819,좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임,0
5820,"세금 내놓으라고 데모질 중 ㅋㅋ간첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같...",1
5821,너가 한 말 중에,0
5822,제갈대중 ㅇㅂ,0


In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
chat_data = pd.read_csv('ChatBotData.csv')

chat_data = chat_data.sample(frac=1).reset_index(drop=True)
chat_data

,Q,A,label
0,별거 아닌거에 속상하네,상처들로 우린 많이 예민해져 있으니까요.,1
1,서로 그냥 밑바닥을 보여준것 같아,그게 진짜 사랑이죠.,1
2,지금도 전하지 못하는 한 마디,마음에 묻어두세요.,1
3,많이 늦은시간인데 연락해봐도 괜찮을까?,최대한 예의를 갖춰서 연락해보세요.,2
4,완전 복붙으로 잘하더라고,능력이라면 능력이겠죠.,0
...,...,...,...
11818,낮에는 그나마 괜찮은데,밤에는 힘든가요.,1
11819,화장이 안 받네,각질제거 먼저 하세요.,0
11820,전남친이 자꾸 생각나,세상은 넓고 사람은 많아요.,0
11821,좋아하는 사람을 원망하게 될 것 같아.,원망은 아픔만 남길 뿐이에요. 털어버리세요.,2


# 텍스트 전처리

**전처리 함수 정의**

In [ ]:
!pip install soynlp
!pip install emoji==1.7.0

In [8]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

In [9]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
special_symbol = re.compile(
    r'([.,?!/@$%~％·∼()\x21-\x2F\x3A-\x40\x5B-\x60\x7B-\x7E])\1{1,}')

In [10]:
def clean(x):
    x = pattern.sub(' ', x)                     # 일반적으로 사용하는 특수문자, 영어, 한글, emoji제외 공백으로 치환
    x = url_pattern.sub('', x)                  # URL 제거
    x = special_symbol.sub('\\1'*1, x)          # 반복되는 특수문자의 축약 횟수 1개로 줄임
    x = x.strip()                               # 문자의 시작과 끝에서 공백제거
    x = repeat_normalize(x, num_repeats=2)      # 반목되는 문자의 축약 횟수 2개로 줄임
    return x

**전처리 및 데이터 분할**

텍스트 전처리는 비속어 감지 데이터에만 적용한다. (챗봇 데이터와 달리 댓글의 성격을 띄기 떄문에 무의미한 반복의 텍스트가 존재하기 때문)

In [11]:
import pandas as pd

In [12]:
# 비속어 감지 데이터
# train : validation = 3 : 1 (5824 = 2^6 * 7 * 13)

train_curse_text = [clean(curse_data['text'][idx]) for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_text = [clean(curse_data['text'][idx]) for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

train_curse_label = [curse_data['label'][idx] for idx in range(0, int((curse_data.shape[0]/4)*3))]
val_curse_label = [curse_data['label'][idx] for idx in range(int((curse_data.shape[0]/4)*3), int((curse_data.shape[0]/4)*4))]

In [13]:
# 챗봇 데이터
# train : validation = 5 : 2 (11823 = 3 * 7 * 563)

train_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (0, int((chat_data.shape[0]/7)*5)))
val_chat_data = pd.DataFrame(chat_data.iloc[idx] for idx in range (int((chat_data.shape[0]/7)*5), int((chat_data.shape[0]/7)*7)))

# 데이터 구축, 토크나이징

**KoGPT2 스페셜토큰 확인**

In [14]:
for i in range (10):
    print("index : ",i," =  tokens : ",GPT2_tokenizer.decode(i))

index :  0  =  tokens :  <s>
index :  1  =  tokens :  </s>
index :  2  =  tokens :  <usr>
index :  3  =  tokens :  <pad>
index :  4  =  tokens :  <sys>
index :  5  =  tokens :  <unk>
index :  6  =  tokens :  <mask>
index :  7  =  tokens :  <d>
index :  8  =  tokens :  </d>
index :  9  =  tokens :  <unused0>


**비속어 감지 데이터**

In [15]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [16]:
# 토크나이징

train_input_token = ELECTRA_tokenizer(train_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")
val_input_token = ELECTRA_tokenizer(val_curse_text, truncation=True, padding=True, max_length=256, return_tensors="pt")

In [17]:
# 비속어 감지 데이터셋

class CurseDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [18]:
train_curse_dataset = CurseDataset(train_input_token, train_curse_label)
val_curse_dataset = CurseDataset(val_input_token, val_curse_label)

In [19]:
# 데이터로더

train_curse_loader = DataLoader(train_curse_dataset, shuffle=True, batch_size=8)
val_curse_loader = DataLoader(val_curse_dataset, shuffle=True, batch_size=8)

**챗봇 데이터**

In [20]:
# 챗봇 데이터셋 (토크나이징 포함)

class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=64):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"           # 새로 추가된 special token을 사용
        self.a_token = "<sys>"
        self.bos = GPT2_tokenizer.bos_token
        self.eos = GPT2_tokenizer.eos_token
        self.mask = GPT2_tokenizer.mask_token
        self.tokenizer = GPT2_tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        index = self._data.iloc[idx]

        q = index["Q"]  # 질문
        q_toked = self.tokenizer.tokenize(self.bos + self.q_token + q)      #   질문
        q_len = len(q_toked)

        a = index["A"]  # 답변
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)      #  답 
        a_len = len(a_toked)

        # 질문의 길이가 최대길이보다 클때
        if q_len > self.max_len: 
            q_toked = q_toked[-(int(self.max_len / 2)):]   # 질문길이를 최대길이의 반으로 
            q_len = len(q_toked)

        # 질문 + 답변 길이가 최대길이보다 클때
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        # 답변의 길이 = 최대길이 - 질문길이

            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              # 답변의 길이를 최대길이 - 질문길이
                
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 질문 + 답변 토큰을 index로 변환   
        token = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 padding
        while len(token) < self.max_len:
            token += [self.tokenizer.pad_token_id]

        # attention(어텐션마스크) = 질문+답변 길이 1 + 나머지(패딩) 0
        attention = [1]*(q_len+a_len) + [0]*(self.max_len - q_len - a_len)

        # token_type_ids(세그먼트 정보) = 질문 0 + 답변 1 + 나머지 0
        token_type = [0]*q_len + [1]*a_len + [0]*(self.max_len - q_len - a_len)

        label = q_toked[0:2] + [self.mask,]*(q_len-2) + a_toked[0:]
        # label을 index로 변환
        label = self.tokenizer.convert_tokens_to_ids(label)
        # 최대길이만큼 padding
        while len(label) < self.max_len:
            label += [self.tokenizer.pad_token_id]

        
        # 질문 + 답변, 어텐션마스크, 세그먼트 정보, 답변
        return (token, attention, token_type, label)

In [21]:
train_chat_dataset = ChatbotDataset(train_chat_data, max_len=64)
val_chat_dataset = ChatbotDataset(val_chat_data, max_len=64)

In [22]:
# collate_fn 구성

def collate_batch(batch):
    token_ids = [item[:][0] for item in batch]
    attention_mask = [item[:][1] for item in batch]
    token_tpye_ids = [item[:][2] for item in batch]
    label_ids = [item[:][3] for item in batch]

    return torch.LongTensor(token_ids), torch.LongTensor(attention_mask), torch.LongTensor(token_tpye_ids), torch.LongTensor(label_ids)

In [23]:
# 데이터로더

train_chat_loader = DataLoader(train_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)
val_chat_loader = DataLoader(val_chat_dataset, shuffle=True, collate_fn = collate_batch, batch_size=16)

# 모델 학습

**모델 파라미터 설정**

In [24]:
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm as tqdm_auto
from sklearn.metrics import accuracy_score

In [25]:
# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
# KcELECTRA 파라미터 설정
ELECTRA_epochs = 5
ELECTRA_learning_rate = 5e-5

ELECTRA_optimizer = torch.optim.AdamW(ELECTRA_model.parameters(), lr=ELECTRA_learning_rate)
ELECTRA_criterion = torch.nn.CrossEntropyLoss()

ELECTRA_step = 0
ELECTRA_eval_steps = len(train_curse_loader)        # 훈련 배치수

In [27]:
# KoGPT2 파라미터 설정
GPT2_epoch = 10
GPT2_learning_rate = 1e-4

GPT2_optimizer = torch.optim.AdamW(GPT2_model.parameters(), lr=GPT2_learning_rate)

GPT2_step = 0
GPT2_eval_steps = len(train_chat_loader)        # 훈련 배치수

In [28]:
ELECTRA_model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [29]:
GPT2_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

**KcELECTRA 학습 진행**

In [30]:
for epoch in range(ELECTRA_epochs):
    n = 0
    train_accuracy = 0
    loss = 0
    train_loss = 0.0
    
    ELECTRA_model.train()
    for batch in tqdm_auto(train_curse_loader, mininterval=0.01, leave=True):
        ELECTRA_optimizer.zero_grad()     # 그래디언트 초기화

        # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = ELECTRA_model(**inputs).logits    # 모형으로 결과 예측

        loss = ELECTRA_criterion(outputs, labels)
        train_loss += loss
        
        loss.backward()
        ELECTRA_optimizer.step()

        # eval_steps 마다 loss를 출력
        ELECTRA_step += 1
        if ELECTRA_step % ELECTRA_eval_steps == 0:
            i = 0
            val_accuracy = 0

            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                ELECTRA_model.eval()        # 평가모드로 전환

                for val_batch in tqdm_auto(val_curse_loader, mininterval=0.01, leave=True):

                    # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
                    val_labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
                    val_outputs = ELECTRA_model(**inputs).logits     # 모형으로 결과 예측

                    loss = ELECTRA_criterion(val_outputs, val_labels)  
                    val_loss += loss

                    val_accuracy += accuracy_score(val_labels.cpu(), val_outputs.argmax(dim=1).cpu())
                    i += 1

                avg_val_loss = val_loss / len(val_curse_loader)

            val_accuracy /= i
            print('Step %d, validation loss: %.4f, accuracy_score: %.2f' % (ELECTRA_step, avg_val_loss, val_accuracy))
            
        avg_train_loss = train_loss / len(train_curse_loader)
        train_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
        n += 1

    train_accuracy /= n
    print('epoch %d, train loss: %.4f, accuracy_score: %.2f \n' % (epoch, avg_train_loss, train_accuracy))

  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

Step 546, validation loss: 0.0396, accuracy_score: 1.00
epoch 0, train loss: 0.3820, accuracy_score: 0.84 



  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

...

  0%|          | 0/182 [00:00<?, ?it/s]

Step 2184, validation loss: 0.0306, accuracy_score: 1.00
epoch 3, train loss: 0.0938, accuracy_score: 0.97 



  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

Step 2730, validation loss: 0.0072, accuracy_score: 1.00
epoch 4, train loss: 0.0799, accuracy_score: 0.98 



**KoGPT2 학습 진행**

In [31]:
# GPU 캐시 비우기 (GPU 메모리 확보)
torch.cuda.empty_cache()

In [32]:
for epoch in range(GPT2_epoch):
    loss = 0
    train_loss = 0.0
    
    GPT2_model.train()
    for batch_idx, samples in enumerate(tqdm_auto(train_chat_loader, mininterval=0.01, leave=True)):
        GPT2_optimizer.zero_grad()       # optimizer 초기화(Gradient)

        # 모델 입력 텐서 GPU에 올리기
        token_ids, attention_mask, token_type_ids, label_ids = samples
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        label_ids = label_ids.to(device)

        out = GPT2_model(
            input_ids=token_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=label_ids,
            )
        
        loss = out.loss
        loss.backward()
        GPT2_optimizer.step()
        
        train_loss += loss.item()

        # GPU 캐시 비우기 (GPU 메모리 확보)
        torch.cuda.empty_cache()
        
        # eval_steps 마다 loss를 출력
        GPT2_step += 1
        if GPT2_step % GPT2_eval_steps == 0:
            n = 0

            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                GPT2_model.eval()

                for val_batch_idx, val_samples in enumerate(tqdm_auto(val_chat_loader, mininterval=0.01, leave=True)):
                    # 모델 입력 텐서 GPU에 올리기
                    val_token_ids, val_attention_mask, val_token_type_ids, val_label_ids = val_samples
                    val_token_ids = val_token_ids.to(device)
                    val_attention_mask = val_attention_mask.to(device)
                    val_token_type_ids = val_token_type_ids.to(device)
                    val_label_ids = val_label_ids.to(device)

                    val_out = GPT2_model(
                            input_ids=val_token_ids,
                            attention_mask=val_attention_mask,
                            token_type_ids=val_token_type_ids,
                            labels=val_label_ids,
                            )
                    loss = out.loss

                    val_loss += loss.item()
                    torch.cuda.empty_cache()
                    
                val_loss /= val_batch_idx

            print('Step %d, validation loss: %.4f' % (GPT2_step, val_loss))
    
    # GPT2_scheduler.step()
    train_loss /= batch_idx
    print('epoch %d, train loss: %.4f \n' % (epoch, train_loss))

  0%|          | 0/528 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]

Step 528, validation loss: 0.4461
epoch 0, train loss: 0.5364 



  0%|          | 0/528 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]

...

  0%|          | 0/212 [00:00<?, ?it/s]

Step 4752, validation loss: 0.0767
epoch 8, train loss: 0.0432 



  0%|          | 0/528 [00:00<?, ?it/s]

  0%|          | 0/212 [00:00<?, ?it/s]

Step 5280, validation loss: 0.0636
epoch 9, train loss: 0.0364 



**테스트**

In [33]:
import sys

In [41]:
num_curse = 0
print("비속어 3회 입력(경고 3번)시 강제종료 됩니다.\n")

while 1:

    q = input("user > ").strip()
    # quit 입력시 챗봇 종료
    if q == "quit":
        break

    clean_text = clean(q)
    ELEC_input = ELECTRA_tokenizer.encode(clean_text, return_tensors="pt").to(device)

    with torch.no_grad():
        output = ELECTRA_model(ELEC_input).logits

        if output.argmax(dim=1).cpu() == 1:
            num_curse += 1
            if num_curse == 3:
                sys.exit("3회 경고 누적으로 강제종료 됩니다.")
            print("비속어가 감지 되었습니다. %d회 경고입니다.\n" % (num_curse))

        else:
            GPT2_input = GPT2_tokenizer.encode(clean_text, return_tensors="pt").to(device)
            gen_ids = GPT2_model.generate(
                GPT2_input,
                do_sample=True,
                max_length=30,
                top_p=0.5,
                top_k=5,
                repetition_penalty=1.0,
                no_repeat_ngram_size=2,
                temperature=0.5,
            )
    
            generated = GPT2_tokenizer.decode(gen_ids[0])
            generated = generated[generated.index("<sys>")+5 : generated.index("</s>")]
            
            print(f'Chatbot > {generated}')

비속어 3회 입력(경고 3번)시 강제종료 됩니다.

user > 나랑 커피 마실래?
Chatbot >  저랑 한잔 해요.
user > 요새 좀 춥다
Chatbot >  따뜻하게 입으세요.
user > 요새 추워진 것 같아
Chatbot >  얼른 뚫어보아요.
user > 시!!바
비속어가 감지 되었습니다. 1회 경고입니다.

user > 넌 누구야?
Chatbot >  저는 위로해드리는 로봇이에요.
user > 아 버러지 같은게 화나게 하잖아~
비속어가 감지 되었습니다. 2회 경고입니다.

user > quit
